# Numerical parameter identifiability

See the [course by Marisa Eisenberg](https://epimath.org/epid-814-materials/) for an excellent introduction into the topic


<img src="assets/identifiability.png" style="max-height: 300px;">

In [ ]:
import numpy as np

from mxlpy import Model, Simulator, fns, plot
from mxlpy.identify import profile_likelihood

We start with an SIR model, which we use to generate some data (this would usually be experimentally measured data)

In [ ]:
def sir() -> Model:
    return (
        Model()
        .add_variables({"s": 0.9, "i": 0.1, "r": 0.0})
        .add_parameters({"beta": 0.2, "gamma": 0.1})
        .add_reaction(
            "infection",
            fns.mass_action_2s,
            args=["s", "i", "beta"],
            stoichiometry={"s": -1, "i": 1},
        )
        .add_reaction(
            "recovery",
            fns.mass_action_1s,
            args=["i", "gamma"],
            stoichiometry={"i": -1, "r": 1},
        )
    )


data = Simulator(sir()).simulate(100).get_result().unwrap_or_err().variables
_ = plot.lines(data)

We then, for `n` different values of each parameter we are interested in, we 

- draw random samples for the remaining model parameters 
- fit the model to the data (excluding the parameter we are interested in) and note the final error
- visualise the error for each parameter value

The error for a parameter should show a clear minimum around the different values used, otherwise it is not identifiable

In [ ]:
errors_beta = profile_likelihood(
    sir(),
    data=data,
    parameter_name="beta",
    parameter_values=np.linspace(0.2 * 0.5, 0.2 * 1.5, 10),
    n_random=10,
)

fig, ax = plot.lines(errors_beta, legend=False)
ax.set(title="beta", xlabel="parameter value", ylabel="abs(error)")
plot.show()